# 00 - Extraccion de datos mediante API de Mercado Libre

## Importación de librerias

In [3]:
import pandas as pd
import numpy as np
import json
import requests

from datetime import datetime, timedelta, timezone, date
pd.options.display.max_columns=None
from pyprojroot import here


import matplotlib.pyplot as plt
import os
import seaborn as sns


In [4]:
path_save = here() / "data" 

In [5]:
here()

WindowsPath('c:/Users/marti/OneDrive/Escritorio/Tesis/mdc-tesis')

In [6]:
import sys
sys.path.append(here())


In [7]:
here()

WindowsPath('c:/Users/marti/OneDrive/Escritorio/Tesis/mdc-tesis')

In [8]:
sys.path.insert(0, 'c:/Users/marti/OneDrive/Escritorio/Tesis/mdc-tesis/library')
import sys

from extraction import MeliAPI

## Armado del Dataset

Para armar el dataset descargaremos para Mercado Libre Argentina laptops publicadas. La categoría se llama: MLA1652.
La API tiene una restricción que no permite utilizar un offset mayor a 4000. Esto nos permite descargar menos de 4000 publicaciones por loop.
Es por ello que se han armado ciertos grupos para descargar la información por grupo. Esto nos permite tambien etiquetar a las publicaciones con este atributo, 
elemento que no se obtiene descargando la información de la publicación.

<img src="..\Grupos.png" alt="Drawing" style="width: 600px;"/>


Para armar estos grupos necesitamos filtar desde el url mediante los siguientes nombres:  

* installments:  
    + yes  
    + no_interest  
* power_seller:  
    + yes  
    + no  
* shipping_cost:  
    + get_available_filtersfree  
            

http://auth.mercadolibre.com.ar/authorization?response_type=code&client_id=8800815426780367&redirect_uri=https://www.google.com.ar/


In [9]:
meli_api = MeliAPI()

http://auth.mercadolibre.com.ar/authorization?response_type=code&client_id=8800815426780367&redirect_uri=https://www.google.com.ar/
<Response [200]>


### Grupo 1

In [10]:
filters = {"installments":"yes",
           "display_size":'(*-14.1")',
          "shipping_cost":"free"}


In [11]:
maximum = meli_api.get_q_items(filters = filters)
print(f'Encontramos {maximum} resultados para nuestra consulta')


Encontramos 4143 resultados para nuestra consulta


In [12]:
data = meli_api.get_df_list(filters = filters)
data["installment"] = "yes"
data["display_size"] = "hasta 14.1" 
data["shipping_cost"] = "free"

El offst es 4000 se terminará la ejecución.


### Grupo 2

In [13]:
filters = {"installments":"yes",
           "display_size":'[14.1"-17")',
                "shipping_cost":"free"}


In [14]:
maximum = meli_api.get_q_items(filters = filters)
print(f'Encontramos {maximum} resultados para nuestra consulta')


Encontramos 4125 resultados para nuestra consulta


In [15]:
data2 = meli_api.get_df_list(filters = filters)
data2["installment"] = "yes"
data2["display_size"] = "entre 14.1 y 16.9"
data2["shipping_cost"] = "free"

El offst es 4000 se terminará la ejecución.


### Grupo 3

In [19]:
meli_api = MeliAPI()

http://auth.mercadolibre.com.ar/authorization?response_type=code&client_id=8800815426780367&redirect_uri=https://www.google.com.ar/
<Response [200]>


In [20]:
filters = {"installments":"no_interest",
           "display_size":'(*-14.1")',
          "shipping_cost":"free"}


In [21]:
maximum = meli_api.get_q_items(filters = filters)
print(f'Encontramos {maximum} resultados para nuestra consulta')


Encontramos 1303 resultados para nuestra consulta


In [22]:
data3 = meli_api.get_df_list(filters = filters)
data3["installment"] = "no_interest"
data3["display_size"] = "hasta 14.1" 
data3["shipping_cost"] = "free"

### Grupo 4

In [23]:
filters = {"installments":"no_interest",
           "display_size":'[14.1"-17")',
          "shipping_cost":"free"}


In [24]:
maximum = meli_api.get_q_items(filters = filters)
print(f'Encontramos {maximum} resultados para nuestra consulta')


Encontramos 2385 resultados para nuestra consulta


In [25]:
data4 = meli_api.get_df_list(filters = filters)
data4["installment"] = "no_interest"
data4["display_size"] = "entre 14.1 y 16.9"
data4["shipping_cost"] = "free"

________________________

In [26]:
data = pd.concat([data, data2, data3, data4])

In [27]:
data.shape

(11646, 76)

In [28]:
data.title.value_counts()

Notebook Dell Core I3 10ma 8gb 1tb + 128gb Ssd 15.6 Touch       20
Notebook Hp I7 1065g7 10ma 8gb Ssd 256gb 15.6  Windows 10       14
Notebook Enova Intel Core I3 10g 240gb Ssd 8gb Ram 14.1 W10      8
Notebook Gfast Intel Celeron N4000 14 4gb 128gb Ssd Win 10       7
Notebook Lenovo Ideapad Amd Ryzen 5 8gb 256gb Ssd Windows 11     7
                                                                ..
Notebook Dell Latitude 5420                                      1
Hp Pavilion X360 14 9uv81la Intel Core I3                        1
Renovada) Dell Latitude 7480 Intel Core I7-7600u X2 2.8ghz 8     1
Renovada) Latest Dell Xps 13 9360 13.3in Fhd Laptop 8th Gen      1
Cx 23800w                                                        1
Name: title, Length: 8914, dtype: int64

In [29]:
import datetime as dt

anio = dt.datetime.now().year
mes = dt.datetime.now().month
dia = dt.datetime.now().day
hora = dt.datetime.now().hour
minuto = dt.datetime.now().minute


In [30]:
path_save /f"datos_laptops_{anio*10000+mes*100+dia}_{hora}.{minuto}.csv"

WindowsPath('c:/Users/marti/OneDrive/Escritorio/Tesis/mdc-tesis/data/datos_laptops_20220916_10.0.csv')

In [31]:
data.to_csv(path_save / f"datos_laptops_{anio*10000+mes*100+dia}_{hora}.{minuto}.csv", index = False, sep = ";")